In [1]:
#ASSN1903

In [2]:
!pip install duckdb pandas

In [3]:
#import libraries
import duckdb
import pandas as pd

In [5]:
#load CSV files
customers_df = pd.read_csv('customers.csv')
purchases_df = pd.read_csv('purchases.csv')

In [6]:
#check first few rows
print(customers_df.head())
print(purchases_df.head())

      id     name   age                email
0   45.0  Jessica  28.0  charlie@example.com
1  393.0    David  46.0  charlie@example.com
2  721.0      Ian  61.0    alice@example.com
3  679.0    David  70.0    alice@example.com
4  563.0    David  39.0    alice@example.com
   customer_id  product_id  quantity
0         37.0        58.0       1.0
1        996.0        50.0       5.0
2        754.0        84.0       2.0
3        691.0        48.0       6.0
4        828.0        23.0       7.0


In [7]:
#register DataFrames with DuckDB
duckdb.register('customers', customers_df)
duckdb.register('purchases', purchases_df)

In [8]:
## JOIN and GROUP data
query1 = """
SELECT
    c.id AS customer_id,
    c.name,
    c.age,
    SUM(p.quantity) AS total_quantity
FROM customers c
JOIN purchases p
ON c.id = p.customer_id
GROUP BY c.id, c.name, c.age
"""

task1_result = duckdb.sql(query1).df()
task1_result

,customer_id,name,age,total_quantity
0,393.0,David,46.0,4.0
1,662.0,Frank,34.0,20.0
2,297.0,Bob,47.0,9.0
3,579.0,Bob,44.0,4.0
4,649.0,Charlie,61.0,4.0
...,...,...,...,...
619,963.0,Ian,29.0,11.0
620,220.0,Hannah,45.0,32.0
621,644.0,David,36.0,12.0
622,752.0,Charlie,34.0,15.0


In [9]:
## Filter by Quantity and Age
## total_quantity > 5 and age < 30
query2 = f"""
SELECT *
FROM ({query1}) AS sub
WHERE total_quantity > 5 AND age < 30
"""

task2_result = duckdb.sql(query2).df()
task2_result

,customer_id,name,age,total_quantity
0,754.0,Alice,28.0,6.0
1,828.0,Eva,28.0,7.0
2,683.0,Charlie,18.0,6.0
3,276.0,Bob,28.0,6.0
4,385.0,Bob,29.0,12.0
...,...,...,...,...
80,423.0,Frank,18.0,9.0
81,979.0,Eva,24.0,8.0
82,347.0,Hannah,27.0,8.0
83,646.0,Jessica,20.0,26.0


In [10]:
## Identify Customers with Multiple Product Categories
query3 = """
SELECT
    c.id AS customer_id,
    c.name,
    COUNT(DISTINCT p.product_id) AS unique_products
FROM customers c
JOIN purchases p
ON c.id = p.customer_id
GROUP BY c.id, c.name
HAVING COUNT(DISTINCT p.product_id) >= 2
"""

task3_result = duckdb.sql(query3).df()
task3_result

,customer_id,name,unique_products
0,70.0,Frank,2
1,991.0,Ian,2
2,820.0,Hannah,2
3,22.0,Eva,2
4,412.0,David,2
...,...,...,...
234,765.0,Frank,2
235,303.0,Eva,3
236,98.0,Frank,2
237,223.0,Frank,2


In [11]:
## Calculate Total Spending for Customers 30+
query4 = """
SELECT
    c.id AS customer_id,
    c.name,
    SUM(p.quantity) AS total_quantity
FROM customers c
JOIN purchases p
ON c.id = p.customer_id
WHERE c.age >= 30
GROUP BY c.id, c.name
"""

task4_result = duckdb.sql(query4).df()
task4_result

,customer_id,name,total_quantity
0,37.0,Eva,13.0
1,936.0,Frank,20.0
2,683.0,Eva,6.0
3,401.0,Ian,8.0
4,239.0,Ian,2.0
...,...,...,...
469,199.0,Eva,9.0
470,438.0,Alice,1.0
471,443.0,David,4.0
472,477.0,David,19.0


In [12]:
## Export Results & Report
task1_result.to_csv('task1_join_group.csv', index=False)
task2_result.to_csv('task2_filter.csv', index=False)
task3_result.to_csv('task3_multi_category.csv', index=False)
task4_result.to_csv('task4_total_spending.csv', index=False)